In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print("NO")
else:
  print(gpu_info)

Wed Mar 22 23:05:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp /content/drive/MyDrive/CosMx_benchmark_pro/mydata.py ./mydata.py

In [ ]:
!pip install ipdb

In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 58.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 41.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.4/868.4 KB 14.7

In [ ]:
import mydata
import matplotlib.pyplot as plt
import torch_geometric as pyg
import torch.nn as nn
import torch 
from torch_geometric.nn import GCNConv, GATConv
import torch.nn.functional as F
from collections import defaultdict
import numpy as np
from torch_geometric.data import data as D 
from tqdm.notebook import tqdm
import os 
import pandas as pd

In [ ]:


class SingletonMeta(type):
    __instance = None
    def __call__(cls,*args,**kwargs):
        if not cls.__instance:
            cls.__instance = type.__call__(cls,*args,**kwargs)
        return cls.__instance

class HugeLog():

  def __init__(self,fpath="log.md"):

    self.f = open(fpath, 'w')
    self.content = defaultdict(list)

  def write(self,content):
    if type(content) != str:
      self.f.write(str(content)+"\n")
    else:
      self.f.write(content+"\n")
    self.f.flush()

  def appendImage(self, key, img):
    self.content[key].append(img)

  def prefixWrite(self, content, prefix="[info]  "):

    if type(content) != str:
      self.write(prefix + str(content))
    else:
      self.write(prefix + content)


  def info(self,content):

    self.prefixWrite(content, prefix="[info]. ")

  def error(self, content):
    self.prefixWrite(content, prefix="[Error]. ")
    

  def other(self,fun,content):
    self.write(fun(content))
    

In [ ]:

def preprocess_X(X,epsilon=0.00001):
    
    mean = X.mean()
    var  = X.var()
    X = (X - mean) / (var + epsilon)
    return X


def preprocess_Y(Y, weight_list=None):
  if weight_list:
    Y = Y / np.array(weight_list)
  else:
    weight_list = [100] + [50] * 17
    Y = Y / weight_list
  return Y


def preprocess_Y_Propobability_softmax(Y):
  
  temp = Y
  temp = np.exp(temp)
  temp = (temp.T / temp.sum(axis=-1)).T
  return temp


def preprocess_Y_Propobability(Y, theta=0.000001):
  
  temp = Y
  temp = (temp.T / (temp.sum(axis=-1) + theta)).T
  return temp

In [ ]:
def pyg_data_maker(x,y,edge_index, edge_attr, train_mask, val_mask, test_mask,
                   x_preprocess = True, ):
    
    data = D.Data()

    if x_preprocess:
      x = preprocess_X(x)

    y = preprocess_Y_Propobability(y)

    x = torch.tensor(x,dtype=torch.float)
    y = torch.tensor(y,dtype=torch.float)
    edge_index = torch.tensor(edge_index.T,dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    data.x, data.y, data.edge_index, data.edge_attr, data.train_mask, data.val_mask, data.test_mask =  x,y,edge_index, edge_attr, train_mask, val_mask, test_mask
    return data

In [ ]:
def fake_graph_generate(nodes, nodes_shape):
  
  X = np.random.rand(nodes, nodes_shape)
  adj, adj_weight = mydata.get_adj(X,topk=5)
  return X, adj, adj_weight

In [ ]:
# model


class GCN(nn.Module):
    def __init__(self, nfeat=980, nhid=256, gnn_out=64, mlp_hidden=36, nout=18, dropout=0.1):
        super(GCN, self).__init__()

        self.gc1 = GCNConv(nfeat, nhid)
        self.gc2 = GCNConv(nhid, gnn_out)
        
    def reset_parameters(self):
        self.gc1.reset_parameters()
        self.gc2.reset_parameters()
        # self.mlp.reset_parameters()
        
    def forward(self, data):
        x =  data.x
        adj = data.edge_index
        weight = data.edge_attr
        x = F.leaky_relu(self.gc1(x, adj, weight))
        x = self.gc2(x, adj, weight)
        return x


In [ ]:

def saveResult(prefix):
  if os.path.isdir(prefix):
    pass
  else:
    os.mkdir(prefix)

def savefig(prefix, content=['train','test','val','test-jsd','test-mse',
                             'test-mae','val-jsd','val-mse','val-mae',
                             "train-mae",'train-jsd','train-mse']):
  for item in content:
    fig = plt.figure()
    plt.plot(total_loss_collect[item])
    plt.title(prefix+"/"+item)
    plt.savefig(prefix+"/"+item+".png")


def logrecord(prefix=None):

  content = ['test-jsd','test-mae','test-mse','test-pcc',
            'val-jsd','val-mae', 'val-mse', 'val-pcc',
            'train-jsd', 'train-mae','train-mse', 'train-pcc']
            
  for item in content:
    mylog.info("min " + item + " : " + str(min(total_loss_collect[item][:-2])))
    mylog.info("select" + item + " : " + str(total_loss_collect[item][-1]))


def outputSave(prefix=None, data=None):
  columns = "tumor	fibroblast	macrophage	T CD4 memory	T CD8 memory	plasmablast	B-cell	mast	Treg	endothelial	pDC	T CD4 naive	neutrophil	T CD8 naive	NK	monocyte	epithelial	mDC".split("	")
  data = F.softmax(model(data),dim=1).detach().cpu().numpy()

  columns = columns[-1 * data.shape[-1]:]
  split_idx = [0,800,1600, len(data)]
  for idx in range(len(split_idx)-1):
    df = pd.DataFrame(data[split_idx[idx] : split_idx[idx+1]], columns=columns)
    df.to_csv(prefix+"-predict-{}.csv".format(idx))

In [ ]:

def pcc(p, q):
    p_mean = torch.mean(p)
    q_mean = torch.mean(q)
    p_centered = p - p_mean
    q_centered = q - q_mean
    numerator = torch.sum(p_centered * q_centered)
    denominator = torch.sqrt(torch.sum(p_centered ** 2)) * torch.sqrt(torch.sum(q_centered ** 2))
    pearson_coef = numerator / denominator
    return torch.nan_to_num(pearson_coef)

def pcc(p, q):
    p_mean = torch.mean(p)
    q_mean = torch.mean(q)
    p_centered = p - p_mean
    q_centered = q - q_mean
    numerator = torch.sum(p_centered * q_centered)
    denominator = torch.sqrt(torch.sum(p_centered ** 2)) * torch.sqrt(torch.sum(q_centered ** 2))
    pearson_coef = numerator / denominator
    return torch.nan_to_num(pearson_coef)
    
def jsd(p_output, q_output):
  """
  Function that measures JS divergence between target and output logits:
  """
  KLDivLoss = nn.KLDivLoss(reduction='batchmean')
  log_mean_output = ((p_output + q_output )/2).log()
  jsd_result = (KLDivLoss(log_mean_output, p_output) + KLDivLoss(log_mean_output, q_output))/2
  return torch.nan_to_num(jsd_result)


def mse(p, q):
  return (p - q).pow(2).mean()

def mae(p, q):
  return (p - q).abs().mean() 

score_fns = [jsd, mse, mae, pcc]

# score_fns = [mse, mae]


In [ ]:
def score_fn(input_, target, mask, softmax=True, prefix='test'):
  
  p = input_[mask]

  if softmax:
    p = F.softmax(p, dim=1)

  q = target[mask]

  # print("p:", p)
  # print("q:", q)

  # import ipdb
  # ipdb.set_trace()

  for fn in score_fns:
    total_loss_collect[prefix + "-" +fn.__name__].append(fn(p, q).item())
  return torch.tensor(0)


def loss_fn(input_, target, mask):
  # input_logit = F.softmax(input_[mask],dim=1)
  # target_logit= F.softmax(target[mask],dim=1)
  input_logit = input_[mask]
  target_logit = target[mask]
  loss = F.cross_entropy(input_logit, target_logit)
  return loss 


def test_with_masked_type(test_type=[0,1,2,3]):
  gnn_out= len(test_type)
  model = GCN(980, gnn_out=gnn_out , nout=len(test_type))
  # print(model)
  model = fit_mask(data, model, 2000, lr=lr, weight_decay=weight_decay, test_type=test_type)
  return model

def fit_mask(data, model, epoches, lr=0.0001, weight_decay=0.99, test_type=[0,1,2]):
  
  if torch.cuda.is_available():
    model = model.cuda()
    data = data.cuda()
    
  opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  # print("learning parameters: lr",lr)
  # train 
  for epoch in tqdm(range(epoches)):
    train_mask_fn(data, model, opt, test_type)
    inference(data, model, test_type, prefix='val')
    inference(data, model, test_type, prefix='test')
    epoch_end(model)

  model = project_end(model)
  inference(data, model, test_type, prefix='train')
  inference(data, model, test_type, prefix='val')
  inference(data, model, test_type, prefix='test')
  return model

def epoch_end(model):

  val_list = total_loss_collect['val-mse']
  if len(val_list) > 1:
    if min(val_list) != val_list[-1]:
      pass
    else:
      torch.save(model.state_dict(), "best_model.pt")

  else:
    torch.save(model.state_dict(), "best_model.pt")

def project_end(model):
  model.load_state_dict(torch.load("best_model.pt"))
  return model


def train_mask_fn(data, model, opt, test_type=[0,1,2]):
  model.train()
  train_mask_ = data.train_mask
  label = data.y
  pred = model(data)

  loss = loss_fn(pred, label[:,test_type], train_mask_)
  score= score_fn(pred, label[:,test_type], train_mask_, prefix='train')

  total_loss_collect['train'].append(loss.item())
  opt.zero_grad()
  loss.backward()
  opt.step()


def inference(data, model, test_type=[0,1,2,3], prefix='val'):
  
  model.eval()
  mask = getattr(data, prefix+"_mask")
  label = data.y
  pred = model(data)
  loss = loss_fn(pred, label[:,test_type], mask)
  score= score_fn(pred, label[:,test_type], mask, prefix=prefix)

  total_loss_collect[prefix].append(loss.item())

In [ ]:
mylog = HugeLog(fpath="log3.md")
mylog.write("Data Description: "+"\n All is TopK5, Propobability method : A/A.sum(-1)")
mylog.write("All mask will be used!")


graph1 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep1/new/spot_gene_expression.csv",
                             "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep1/new/ground_truth.csv", topk=5)

graph2 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep2/new/spot_gene_expression.csv",
                             "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep2/new/ground_truth.csv",topk=5)

graph3 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep3/new/spot_gene_expression.csv",
                             "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep3/new/ground_truth.csv",topk=5)

graph4 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung12/new/spot_gene_expression.csv",
                            "/content/drive/MyDrive/deconvo_benchmark/data/Lung12/new/ground_truth.csv",topk=5)

graph5 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung13/new/spot_gene_expression.csv",
                            "/content/drive/MyDrive/deconvo_benchmark/data/Lung13/new/ground_truth.csv",topk=5)

# total_graph = mydata.union_graph([graph1, graph2, graph3, graph4],topk=5)
total_graph = mydata.union_graph([graph1, graph2, graph3],topk=10)

# test_mask, train_mask, val_mask = mydata.mask_graph(total_graph, masked_graph_idx=masked_graph_idx)

lrs = [0.01,0.05,0.001,0.005]
# lrs = [0.01]

weight_decay_list = [0.01, 0.05, 0.001, 0.005, 0.0001]
# weight_decay_list = [0.01]


# masked_jobs = [[2]]
masked_jobs = [[0], [1], [2]]
# masked_jobs = [[3]]

for lr in lrs:
  for masked_graph_idx in masked_jobs:

    test_mask, train_mask, val_mask = mydata.mask_graph(total_graph, masked_graph_idx=masked_graph_idx)

    data = pyg_data_maker(total_graph.X, 
                          total_graph.label, 
                          total_graph.adj_link.T,
                          total_graph.adj_weight,
                          train_mask,
                          val_mask,
                          test_mask,
                          x_preprocess=False)
    
    mask_description = "mask-{}".format("-".join(str(i) for i in masked_graph_idx))

    # job1 

    for weight_decay in weight_decay_list:
      test_type = list(range(18))
      total_loss_collect = defaultdict(list) 

      model = test_with_masked_type(test_type=test_type)

      prefix = "{}-{}-lr-{}-weight-decay-{}".format(mask_description,len(test_type), lr, weight_decay)
      mylog.write("# "+ prefix)
      saveResult(prefix)
      savefig(prefix)
      logrecord()
      outputSave(prefix, data=data)

    # # job2 
    # for weight_decay in weight_decay_list:
    #   test_type = list(range(1,18))
    #   total_loss_collect = defaultdict(list) 

    #   model = test_with_masked_type(test_type=test_type)

    #   prefix = "{}-{}-lr-{}-weight-decay-{}".format(mask_description,len(test_type), lr, weight_decay)
    #   mylog.write("# "+ prefix)
    #   saveResult(prefix)
    #   savefig(prefix)
    #   logrecord()
    #   outputSave(prefix, data=data)

m = open("log3.md",'r').read().split('\n')
content = defaultdict(list)
prefix = None
flag = True
columns = []
for idx in range(len(m)):
  if len(m[idx]) > 1:
    if m[idx][0] == "#":
      if prefix and flag:
        flag = False
      prefix = m[idx][2:]
      if "log2" in prefix:
        prefix = prefix[5:]
    elif prefix :
      if flag:
        columns.append(m[idx].split(":")[0])
      content[prefix].append(m[idx].split(":")[1])
klist = list(content.keys())
for i in klist:
  content[i] = content[i][:26]
df = pd.DataFrame(content).T
df.columns = columns
df.to_excel("res2.xlsx")

In [ ]:
os.mkdir("result") 
!mv *.csv result
!mv *.xlsx result
!mv log3.md result

In [ ]:
!mv result lung_123_5_10
!cp -r ./lung_123_5_10 /content/drive/MyDrive/CosMx_benchmark_pro/overall_result/lung/

In [ ]:
# m = open("log3.md",'r').read().split('\n')
# content = defaultdict(list)
# prefix = None
# flag = True
# columns = []
# for idx in range(len(m)):
#   if len(m[idx]) > 1:
#     if m[idx][0] == "#":
#       if prefix and flag:
#         flag = False
#       prefix = m[idx][2:]
#       if "log2" in prefix:
#         prefix = prefix[5:]
#     elif prefix :
#       if flag:
#         columns.append(m[idx].split(":")[0])
#       content[prefix].append(m[idx].split(":")[1])

# df = pd.DataFrame(content).T
# df.columns = columns
# df.to_excel("res2.xlsx")

In [ ]:
# def outputSave(prefix=None, data=None):
#   columns = "tumor	fibroblast	macrophage	T CD4 memory	T CD8 memory	plasmablast	B-cell	mast	Treg	endothelial	pDC	T CD4 naive	neutrophil	T CD8 naive	NK	monocyte	epithelial	mDC".split("	")
#   data = F.softmax(model(data),dim=1).detach().cpu().numpy()
#   columns = columns[-1 * data.shape[-1]:]
#   split_idx = [0,600,1200, 1800,len(data.y)]
#   for idx in range(len(split_idx)-1):
#     df = pd.DataFrame(data[split_idx[idx] : split_idx[idx+1]], columns=columns)
#     df.to_csv(prefix+"-predict-{}.csv".format(idx))

In [ ]:
# graph1 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep1/new/spot_gene_expression.csv",
#                              "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep1/new/ground_truth.csv", topk=5)
# graph2 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep2/new/spot_gene_expression.csv",
#                              "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep2/new/ground_truth.csv",topk=5)
# graph3 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep3/new/spot_gene_expression.csv",
#                              "/content/drive/MyDrive/deconvo_benchmark/data/Lung5_Rep3/new/ground_truth.csv",topk=5)

# graph4 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung12/new/spot_gene_expression.csv",
#                             "/content/drive/MyDrive/deconvo_benchmark/data/Lung12/new/ground_truth.csv",topk=5)

# graph5 = mydata.single_graph("/content/drive/MyDrive/deconvo_benchmark/data/Lung13/new/spot_gene_expression.csv",
#                             "/content/drive/MyDrive/deconvo_benchmark/data/Lung13/new/ground_truth.csv",topk=5)
# total_graph = mydata.union_graph([graph1, graph2, graph3, graph4],topk=5)

# test_mask, train_mask, val_mask = mydata.mask_graph(total_graph, masked_graph_idx=[3])
# data = pyg_data_maker(total_graph.X, 
#                         total_graph.label, 
#                         total_graph.adj_link.T,
#                         total_graph.adj_weight,
#                         train_mask,
#                         val_mask,
#                         test_mask,
#                         x_preprocess=False)
# split_idx = [0,600,1200, 1800,len(data.y)]
# data.y = data.y.numpy()
# columns = "tumor	fibroblast	macrophage	T CD4 memory	T CD8 memory	plasmablast	B-cell	mast	Treg	endothelial	pDC	T CD4 naive	neutrophil	T CD8 naive	NK	monocyte	epithelial	mDC".split("	")
# for idx in range(len(split_idx)-1):
#   df = pd.DataFrame(data.y[split_idx[idx] : split_idx[idx+1]], columns=columns)
#   df.to_csv("./ground-truth-prob-{}.csv".format(idx))